In [ ]:
import sys

In [ ]:
import numpy as np
import joblib
import pandas as pd
from IPython.display import display
from tqdm import tqdm

In [ ]:
src_dir = "../../src"

In [ ]:
sys.path.append(src_dir)

In [ ]:
from package.constants import *
from package.datasets import *
from package.feature_extraction import *
from package.preprocessing import *

In [ ]:
pd.options.display.max_rows = None

In [ ]:
processed = load_processed()

In [ ]:
model = joblib.load(lgbm_reg_path)

In [ ]:
is_test = (validation_start_date <= processed["date"]) & (
    processed["date"] <= evaluation_end_date
)
dates = processed.loc[is_test, "date"].unique()

In [ ]:
%%time
for date in tqdm(dates[evaluation_days::prediction_step]):
    date_end = date + np.timedelta64(prediction_step - 1, "D")
    is_predicted = (processed["date"] >= date - np.timedelta64(max_lags, "D")) & (
        processed["date"] <= date_end
    )

    intermediate = processed[is_predicted].copy()

    # See below:
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/139461
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/141515
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/141545
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/141743
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/141972
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/151339
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/151614
    # https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/151927
    create_level_targets(intermediate)
    create_shift_features(
        intermediate, level_ids[11], level_targets[9:], periods_online
    )
    create_rolling_features(
        intermediate, level_ids[11:], shift_features_online, windows
    )

    is_predicted = date <= intermediate["date"]
    X = intermediate.loc[is_predicted, features]

    display(X.isnull().sum())

    y_pred = model.predict(X)
    y_pred = y_pred.astype("float32")
    y_pred /= intermediate.loc[is_predicted, "sell_price"]

    # See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/137217
    y_pred = np.maximum(0.0, y_pred)

    is_predicted = (date <= processed["date"]) & (processed["date"] <= date_end)
    processed.loc[is_predicted, target] = y_pred

In [ ]:
processed = processed.loc[is_test, ["d", "id", target]]

In [ ]:
processed.to_parquet(prediction_path)